In [1]:
pip install pillow matplotlib torchvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append('/app/src')

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import itertools
from model import CycleGANGenerator, UVCGANGenerator, Discriminator, ViTUNetGenerator
from datamodule import DataModule
from trainer import Trainer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def experiment_CycleGAN():
    generator_A2B = CycleGANGenerator(3, 3)
    generator_B2A = CycleGANGenerator(3, 3)
    discriminator_A = Discriminator(3)
    discriminator_B = Discriminator(3)

    base_lr = 0.001
    
    config = {
        'datamodule': DataModule(),
        'generator_A2B': generator_A2B,
        'generator_B2A': generator_B2A,
        'discriminator_A': discriminator_A,
        'discriminator_B': discriminator_B,
        'criterion_GAN': nn.MSELoss(),
        'criterion_cycle': nn.L1Loss(),
        'criterion_identity': nn.L1Loss(),
        'optimizer_G': optim.Adam(itertools.chain(generator_A2B.parameters(), generator_B2A.parameters()), lr=base_lr),
        'optimizer_D_A': optim.Adam(discriminator_A.parameters(), lr=4*base_lr),
        'optimizer_D_B': optim.Adam(discriminator_B.parameters(), lr=4*base_lr),
        'max_epoch': 20,
        'output_dir': '../output/cyclegan'
    }
    
    trainer = Trainer(config)
    trainer.fit()

def experiment_UVCGAN():
    generator_A2B = ViTUNetGenerator((3,64,64),[48, 96, 192, 384])
    generator_B2A = ViTUNetGenerator((3,64,64),[48, 96, 192, 384])
    #generator_A2B = UVCGANGenerator(3, 3)
    #generator_B2A = UVCGANGenerator(3, 3)
    discriminator_A = Discriminator(3)
    discriminator_B = Discriminator(3)

    base_lr = 0.001
    
    config = {
        'datamodule': DataModule(),
        'generator_A2B': generator_A2B,
        'generator_B2A': generator_B2A,
        'discriminator_A': discriminator_A,
        'discriminator_B': discriminator_B,
        'criterion_GAN': nn.MSELoss(),
        'criterion_cycle': nn.L1Loss(),
        'criterion_identity': nn.L1Loss(),
        'optimizer_G': optim.Adam(itertools.chain(generator_A2B.parameters(), generator_B2A.parameters()), lr=base_lr),
        'optimizer_D_A': optim.Adam(discriminator_A.parameters(), lr=4*base_lr),
        'optimizer_D_B': optim.Adam(discriminator_B.parameters(), lr=4*base_lr),
        'max_epoch': 20,
        'output_dir': '../output/uvcgan'
    }
    
    trainer = Trainer(config)
    trainer.fit()

In [5]:
#experiment_CycleGAN()
experiment_UVCGAN()

/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([8, 3, 64, 64])
torch.Size([8, 16, 384])
Shape of embed: torch.Size([8, 256, 384])
Shape of x: torch.Size([8, 16, 384])


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 256 but got size 16 for tensor number 1 in the list.